In [ ]:
#!/usr/bin/python

# ... import files
from sys import stdout, argv
from time import *
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib
import math
import os
import scipy.stats
import random
import SimpleITK as sitk
import pydicom
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
import cv2  
from PIL import Image
import radiomics
from radiomics import featureextractor
import logging
import pywt
from matplotlib import pyplot as plt
from pywt._doc_utils import wavedec2_keys, draw_2d_wp_basis
from skimage.feature import peak_local_max
import pandas as pd

#startTime = clock()
#ctime1    = time()

In [ ]:
## Path acquisition for "in vitro" (DRR)
# Define paths
root_path = os.getcwd().replace("\\", "/") + "/In_vitro/"
print(root_path)
out_path = os.getcwd().replace("\\", "/") + "/Processing/CT/"
print(out_path)

profile_paths = []
face_paths = []
patients = []
# Store file paths
for subdir, dirs, files in os.walk(root_path):
    dirs[:] = [d for d in dirs if d not in ['Incomplete']]
    for direct in dirs:
        if ("PA" in direct) and ("PA18043" not in direct) and ("PA18044" not in direct) and ("PA18045" not in direct) and ("PA18046" not in direct) and ("PA18047" not in direct) and ("PA18048" not in direct) and ("PA20013" not in direct) and ("PA20014" not in direct) and ("PA17015" not in direct) and ("PA17033" not in direct) and ("PA17039" not in direct) and ("PA17040" not in direct):# and ("PA20007" not in direct) and ("PA20009" not in direct) and ("PA20010" not in direct) ||| ("PA14" in direct) or ("PA17007" in direct) or ("PA17009" in direct) or ("PA17010" in direct) or ("PA17013" in direct) or ("PA17014" in direct):
            patients.append(direct)

face_paths =[]; profile_paths = []
profile_L1_paths = []; profile_L2_paths = []; profile_L3_paths = []; profile_L4_paths = []

face_L1_paths = []; face_L2_paths = []; face_L3_paths = []; face_L4_paths = []
for patient in patients:
    directory = root_path + str(patient) + "/DRR/"
    if (patient == "PA17007") or  (patient == "PA17009") or (patient == "PA17010") or (patient == "PA17013") or (patient == "PA17014") :
        path = directory + "face_DRR_L1_sans_TM_ref.dcm"
        if os.path.exists(path):
            face_paths.append(path)
        path = directory + "profil_DRR_L1_sans_TM_ref.dcm"
        if os.path.exists(path):
            profile_paths.append(path)
    else:
        path = directory + "face_DRR_L1_ref.dcm"
        if os.path.exists(path):
            face_paths.append(path)    
        path = directory + "profil_DRR_L1_ref.dcm"
        if os.path.exists(path):
            profile_paths.append(path)
    path = directory + "masque_ap_L1.tiff"
    if os.path.exists(path):
        face_L1_paths.append(path)
    else: face_L1_paths.append(" ")
    path = directory + "masque_ap_L2.tiff"
    if os.path.exists(path):
        face_L2_paths.append(path)
    else: face_L2_paths.append(" ")
    path = directory + "masque_ap_L3.tiff"
    if os.path.exists(path):
        face_L3_paths.append(path)
    else: face_L3_paths.append(" ")
    path = directory + "masque_ap_L4.tiff"
    if os.path.exists(path):
        face_L4_paths.append(path)
    else: face_L4_paths.append(" ")
    path = directory + "masque_lat_L1.tiff"
    if os.path.exists(path):
        profile_L1_paths.append(path)
    else: profile_L1_paths.append(" ")
    path = directory + "masque_lat_L2.tiff"
    if os.path.exists(path):
        profile_L2_paths.append(path)
    else: profile_L2_paths.append(" ")
    path = directory + "masque_lat_L3.tiff"
    if os.path.exists(path):
        profile_L3_paths.append(path)
    else: profile_L3_paths.append(" ")
    path = directory + "masque_lat_L4.tiff"
    if os.path.exists(path):
        profile_L4_paths.append(path)
    else: profile_L4_paths.append(" ")
    #Create processing directories
    if not os.path.exists(out_path + patient):
            os.makedirs(out_path + patient)

In [ ]:
## Extract Dicom images
face_images = []
profile_images = []
print(len(face_paths))
print(len(profile_paths))
for i in range(0,len(face_paths)):
    # specify your image path
    face_path = face_paths[i]
    profile_path = profile_paths[i]
    # read dicom image
    face_image = pydicom.dcmread(face_path)
    profile_image = pydicom.dcmread(profile_path)
    #convert to array
    face_image = face_image.pixel_array
    profile_image = profile_image.pixel_array
    face_images.append(face_image)
    profile_images.append(profile_image)

In [ ]:
## Extract ROI (biggest inner rectangle located inside trabecular bone mask) (DRR)
## Only work with profile images
im_face_crop_small_path = np.empty((4,len(face_paths)),dtype=np.dtype('U1000'))
im_profile_crop_small_path = np.empty((4,len(profile_paths)),dtype=np.dtype('U1000'))
for i in range(0,len(face_paths)):
    im = face_images[i]
    print(i)
    for j in range(1,5): # number of vertebrae
        if j == 1 and face_L1_paths[i] != " ":
            print(face_L1_paths[i])
            mask = cv2.imread(face_L1_paths[i],0)
        elif j == 2 and face_L2_paths[i] != " ":
            print(face_L2_paths[i])
            mask = cv2.imread(face_L2_paths[i],0)
        elif j == 3 and face_L3_paths[i] != " ":
            print(face_L3_paths[i])
            mask = cv2.imread(face_L3_paths[i],0)
        elif j == 4 and face_L4_paths[i] != " ":
            print(face_L4_paths[i])
            mask = cv2.imread(face_L4_paths[i],0)
        else: continue;# Select the contour
        #plt.imshow(mask)
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
        # if your mask is incurved or if you want better results, 
        # you may want to use cv2.CHAIN_APPROX_NONE instead of cv2.CHAIN_APPROX_SIMPLE, 
        # but the rectangle search will be longer
        im_copy = im.copy()

        cv2.drawContours(im_copy, contours, -1, (255,0,0), 1)
        cv2.waitKey(0)

        # Get all the points of the contour
        contour = contours[0].reshape(len(contours[0]),2)
        # we assume a rectangle with at least two points on the contour gives a 'good enough' result
        # get all possible rectangles based on this hypothesis
        rect = []

        for k in range(len(contour)):
            x1, y1 = contour[k]
            for l in range(len(contour)):
                x2, y2 = contour[l]
                area = abs(y2-y1)*abs(x2-x1)
                rect.append(((x1,y1), (x2,y2), area))

        # the first rect of all_rect has the biggest area, so it's the best solution if he fits in the picture
        all_rect = sorted(rect, key = lambda x : x[2], reverse = True)

        # we take the largest rectangle we've got, based on the value of the rectangle area
        # only if the border of the rectangle is not in the black part
        
        # if the list is not empty
        if all_rect:

            best_rect_found = False
            index_rect = 0
            nb_rect = len(all_rect)

            # we check if the rectangle is  a good solution
            while not best_rect_found and index_rect < nb_rect:

                rect = all_rect[index_rect]
                (x1, y1) = rect[0]
                (x2, y2) = rect[1]

                valid_rect = True

                # we search a black area in the perimeter of the rectangle (vertical borders)
                x = min(x1, x2)
                while x <max(x1,x2)+1 and valid_rect:
                    if mask[y1,x] == 0 or mask[y2,x] == 0:
                        # if we find a black pixel, that means a part of the rectangle is black
                        # so we don't keep this rectangle
                        valid_rect = False
                    x+=1

                y = min(y1, y2)
                while y <max(y1,y2)+1 and valid_rect:
                    if mask[y,x1] == 0 or mask[y,x2] == 0:
                        valid_rect = False
                    y+=1

                if valid_rect:
                    best_rect_found = True

                index_rect+=1
            print(best_rect_found)
            if best_rect_found:

                cv2.rectangle(im_copy, (x1,y1), (x2,y2), (255,0,0), 1)
                #cv2.imshow(im_copy)
                cv2.waitKey(0)

                # Finally, we crop the picture and store it
                result = im[min(y1+1, y2):max(y1+1, y2), min(x1+1,x2):max(x1+1,x2)]
                
                im_face_crop_small_path[j-1,i] = out_path + patients[i] + "/face_L" + str(j) + "_crop_small.png"
                print(im_face_crop_small_path[j-1,i])
                #cv2.imshow(result)
                #cv2.imwrite(im_face_crop_small_path[j-1,i],result)
                #cv2.waitKey(0)
            else:
                print("No rectangle fitting into the area")

        else:
            print("No rectangle found")
    im = profile_images[i]
    for j in range(1,5): # number of vertebrae
        if j == 1 and profile_L1_paths[i] != " ":
            mask = cv2.imread(profile_L1_paths[i],0)
        elif j == 2 and profile_L2_paths[i] != " ":
            mask = cv2.imread(profile_L2_paths[i],0)
        elif j == 3 and profile_L3_paths[i] != " ":
            mask = cv2.imread(profile_L3_paths[i],0)
        elif j == 4 and profile_L4_paths[i] != " ":
            mask = cv2.imread(profile_L4_paths[i],0)
        else: continue;# Select the contour
        #plt.imshow(mask)
        
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
        # if your mask is incurved or if you want better results, 
        # you may want to use cv2.CHAIN_APPROX_NONE instead of cv2.CHAIN_APPROX_SIMPLE, 
        # but the rectangle search will be longer
        im_copy = im.copy()

        cv2.drawContours(im_copy, contours, -1, (255,0,0), 1)
        cv2.waitKey(0)

        # Get all the points of the contour
        contour = contours[0].reshape(len(contours[0]),2)

        # we assume a rectangle with at least two points on the contour gives a 'good enough' result
        # get all possible rectangles based on this hypothesis
        rect = []

        for k in range(len(contour)):
            x1, y1 = contour[k]
            for l in range(len(contour)):
                x2, y2 = contour[l]
                area = abs(y2-y1)*abs(x2-x1)
                rect.append(((x1,y1), (x2,y2), area))

        # the first rect of all_rect has the biggest area, so it's the best solution if he fits in the picture
        all_rect = sorted(rect, key = lambda x : x[2], reverse = True)

        # we take the largest rectangle we've got, based on the value of the rectangle area
        # only if the border of the rectangle is not in the black part

        # if the list is not empty
        if all_rect:

            best_rect_found = False
            index_rect = 0
            nb_rect = len(all_rect)

            # we check if the rectangle is  a good solution
            while not best_rect_found and index_rect < nb_rect:

                rect = all_rect[index_rect]
                (x1, y1) = rect[0]
                (x2, y2) = rect[1]

                valid_rect = True

                # we search a black area in the perimeter of the rectangle (vertical borders)
                x = min(x1, x2)
                while x <max(x1,x2)+1 and valid_rect:
                    if mask[y1,x] == 0 or mask[y2,x] == 0:
                        # if we find a black pixel, that means a part of the rectangle is black
                        # so we don't keep this rectangle
                        valid_rect = False
                    x+=1

                y = min(y1, y2)
                while y <max(y1,y2)+1 and valid_rect:
                    if mask[y,x1] == 0 or mask[y,x2] == 0:
                        valid_rect = False
                    y+=1

                if valid_rect:
                    best_rect_found = True

                index_rect+=1
            print(best_rect_found, flush=True)
            cv2.waitKey(2)
            if best_rect_found:

                cv2.rectangle(im_copy, (x1,y1), (x2,y2), (255,0,0), 1)
                #cv2.imshow(im_copy)
                cv2.waitKey(0)

                # Finally, we crop the picture and store it
                result = im[min(y1+1, y2):max(y1+1, y2), min(x1+1,x2):max(x1+1,x2)]
            
                im_profile_crop_small_path[j-1,i] = out_path + patients[i] + "/profil_L" + str(j) + "_crop_small.png"
                #cv2.imshow(result)
                print(im_profile_crop_small_path[j-1,i], flush=True)
                cv2.imwrite(im_profile_crop_small_path[j-1,i],result)
                #cv2.waitKey(0)
            else:
                print("No rectangle fitting into the area")

        else:
            print("No rectangle found")

In [ ]:
from skimage import exposure

## Equalization of histogram
im_crop_equalized_path = np.empty((4,len(face_paths)),dtype=np.dtype('U1000'))
im_crop_small_path = im_profile_crop_small_path
for i in range(0,im_crop_small_path.shape[1]):
    print(i)
    for j in range(0,4):
        if im_crop_small_path[j,i] == "": continue;
        else:
            im = cv2.imread(im_crop_small_path[j,i],0)
            im_eq = exposure.equalize_hist(im)
            im_crop_equalized_path[j,i] = out_path + patients[i] + "/profil_L" + str(j+1) + "_crop_small_equalized.png"
            #cv2.imshow(result)
            im_eq = cv2.convertScaleAbs(im_eq, alpha=(255.0))
            #plt.hist(im_eq.ravel())
            cv2.imwrite(im_crop_equalized_path[j,i],im_eq)

In [ ]:
## (OPTIONAL) Imports for new data with csv filename

## Path acquisition for "in vitro" (DRR)
# Define paths
root_path = os.getcwd().replace("\\", "/") + "/Unprocessed data/"

df = pd.read_csv(root_path + "DRR_filenames_labels_arbitrary_simple.csv", sep=";")
file_paths = df["file_name"].values
out_path = os.getcwd().replace("\\", "/") + "/New features/"

patients = np.arange(0,48)
profile_L1_paths = np.empty(len(patients),dtype=np.dtype('U1000'))
profile_L2_paths = np.empty(len(patients),dtype=np.dtype('U1000'))
profile_L3_paths = np.empty(len(patients),dtype=np.dtype('U1000'))
profile_L4_paths = np.empty(len(patients),dtype=np.dtype('U1000'))
k = -1
for patient in patients:
    k = k +1
    if patient < 10:
        p = "0" +str(patient)
    else: p = str(patient)
    path = root_path + p + "_profil_L1_crop_small.png"
    if os.path.exists(path):
        profile_L1_paths[k] = path
    else: profile_L1_paths[k] = " "
    path = root_path + p + "_profil_L2_crop_small.png"
    if os.path.exists(path):
        profile_L2_paths[k] = path
    else: profile_L2_paths[k] = " "
    path = root_path + p + "_profil_L3_crop_small.png"
    if os.path.exists(path):
        profile_L3_paths[k] = path
    else: profile_L3_paths[k] = " "
    path = root_path + p + "_profil_L4_crop_small.png"
    if os.path.exists(path):
        profile_L4_paths[k] = path
    else: profile_L4_paths[k] = " "

#Creation of array with all paths        
im_profile_crop_small_path = np.empty((4,len(patients)),dtype=np.dtype('U1000'))
for i in range(0,len(patients)):
    for j in range(0,4):
        if j==0 and profile_L1_paths[i] != " " and profile_L1_paths[i] != "":
            im_profile_crop_small_path[j,i] = profile_L1_paths[i]
        elif j==1 and profile_L2_paths[i] != " " and profile_L2_paths[i] != "":
            im_profile_crop_small_path[j,i] = profile_L2_paths[i]
        elif j==2 and profile_L3_paths[i] != " " and profile_L3_paths[i] != "":
            im_profile_crop_small_path[j,i] = profile_L3_paths[i]
        elif j==3 and profile_L4_paths[i] != " " and profile_L4_paths[i] != "":
            im_profile_crop_small_path[j,i] = profile_L4_paths[i]
        else:
            im_profile_crop_small_path[j,i] = " "

In [ ]:
# computation of Fourier features
im_crop_small_path = im_profile_crop_small_path# use im_crop_equalized_path when using the raw data

df_fourier = pd.DataFrame(columns={"id","Patient","Vertebrae","RMS","FMS"})
# Compute Fourier Texture Features
def rms_flat(a):
    """
    Return the root mean square of all the elements of *a*, flattened out.
    """
    return np.sqrt(np.mean(np.absolute(a)**2))


def rms_fft(spectrum):
    """
    Use Parseval's theorem to find the RMS value of a signal from its fft,
    without wasting time doing an inverse FFT.
    For a signal x, these should produce the same result, to within numerical
    accuracy:
    rms_flat(x) ~= rms_fft(fft(x))
    """
    return rms_flat(spectrum)/np.sqrt(len(spectrum))

rms=np.zeros((4,im_crop_small_path.shape[1]))
fms=np.zeros((4,im_crop_small_path.shape[1]))
for i in range(0,im_crop_small_path.shape[1]):
    for j in range(0,4):
        if (im_crop_small_path[j,i] == " ") or (im_crop_small_path[j,i] == ""): continue;
        else:
            print(im_crop_small_path[j,i])
            im = cv2.imread(im_crop_small_path[j,i],0)
            f = np.fft.fft2(im)
            fshift = np.fft.fftshift(f)
            spectrum = 20*np.log(np.abs(fshift))
            rms[j,i] = rms_fft(spectrum)
            #print("RMS: ", rms[j,i])
            fms[j,i] = np.sum(abs(spectrum)) / len(spectrum)
            #fms = scipy.stats.moment(spectrum, 1)
            #print("FMS: ", fms[j,i])
            df2 = {'id': int(i+1),'Patient': patients[i], 'Vertebrae': "L" + str(j+1), 'FMS': fms[j,i], 'RMS': rms[j,i]}
            df_fourier = df_fourier.append(df2, ignore_index = True)

cols = ["id","Patient","Vertebrae","FMS","RMS"]
excel_path = out_path + "profile_fourier_features.xlsx"
df_fourier.to_excel(excel_path, columns=cols)

In [ ]:
from cv2 import waitKey

out_path = root_path + "Processing/"

# Wavelet computation
im_crop_small_path = im_profile_crop_small_path
wavelets=np.zeros((4,len(patients),7))
fms=np.zeros((4,len(patients)))
im_wavelet_paths = np.empty((4,len(patients),7),dtype=np.dtype('U1000'))

for i in range(0,im_crop_small_path.shape[1]):
    if not os.path.exists(out_path + str(patients[i]) + "/wavelets"):
            os.makedirs(out_path + str(patients[i]) + "/wavelets")
    for j in range(0,4):
        if (im_crop_small_path[j,i] == " ") or (im_crop_small_path[j,i] == ""): continue;
        else:
            x = cv2.imread(im_crop_small_path[j,i],0)
            shape = x.shape

            max_lev = 2       # how many levels of decomposition to draw
            label_levels = 2  # how many levels to explicitly label on the plots
            im_wavelet_path = []
            fig, axes = plt.subplots(2, 4, figsize=[14, 8])
            arr3 = []
            for level in range(max_lev, max_lev + 1):
                if level == 0:
                    # show the original image before decomposition
                    axes[0, 0].set_axis_off()
                    axes[1, 0].imshow(x, cmap=plt.cm.gray)
                    axes[1, 0].set_title('Image')
                    axes[1, 0].set_axis_off()
                    continue

                # plot subband boundaries of a standard DWT basis
                draw_2d_wp_basis(shape, wavedec2_keys(level), ax=axes[0, level],
                                 label_levels=label_levels)
                axes[0, level].set_title('{} level\ndecomposition'.format(level))

                # compute the 2D DWT
                c = pywt.wavedec2(x, 'db2', mode='periodization', level=level)
                
                # normalize each coefficient array independently for better visibility
                c[0] /= np.abs(c[0]).max()
                for detail_level in range(level):
                    c[detail_level + 1] = [d/np.abs(d).max() for d in c[detail_level + 1]]
                    
                # show the normalized coefficients
                arr, slices = pywt.coeffs_to_array(c)
                axes[1, level].imshow(arr, cmap=plt.cm.gray)
                axes[1, level].set_title('Coefficients\n({} level)'.format(level))
                axes[1, level].set_axis_off()
                arr2, slices = pywt.coeffs_to_array(c)
                arr3.append(arr2[slices[0]]*255)
                arr3.append(arr2[slices[1]['ad']]*255)
                arr3.append(arr2[slices[1]['da']]*255)
                arr3.append(arr2[slices[1]['dd']]*255)
                if level == 2:
                  arr3.append(arr2[slices[2]['ad']]*255)
                  arr3.append(arr2[slices[2]['da']]*255)
                  arr3.append(arr2[slices[2]['dd']]*255)
                
                for k in range(0, 7):
                    im_wavelet_paths[j,i,k] = out_path + str(patients[i]) + "/wavelets/profile_L" + str(j) + "_im_wavelet_level_" + str(level) +"_pos_"+ str(k) + "_eq.png"
                    #sitk.WriteImage(arr3[k], im_wavelet_paths[j,i,k])
                    #save wavelet images
                    cv2.imwrite(im_wavelet_paths[j,i,k],arr3[k])
                    #waitKey(0)
                plt.tight_layout()
                plt.show()

In [ ]:
## Wavelet features computation (GLCM, GLDM, GLRLM & GLSZM )

wavelet_feature_vector = np.zeros((4,len(patients),7))
mask_paths = np.empty((4,len(patients),7),dtype=np.dtype('U1000'))
l=-1

# Important, run once commenting these first few lines containing df_test first, then uncomment them and run again
# this is to obtain the column names of all the features
# code will be interrupted because of df_test not existing in the loop on the first run (this is okay)
df_test = df[0:0]
df_test["Wavelet"] = ""
df_test["Vertebrae"] = ""
df_test["Patient"] = ""
df_test["index"] = ""
df_test.set_index("index")
cols = df_test.columns
cols = cols[-4:].append(cols[:-4])
df_test = df_test[cols]

for i in range(0,im_wavelet_paths.shape[1]):
    l = l +1
    if not os.path.exists(out_path + str(patients[i]) + "/wavelets/masks"):
            os.makedirs(out_path + str(patients[i]) + "/wavelets/masks")
    for j in range(0,im_wavelet_paths.shape[0]):
        if (im_crop_small_path[j,i] == " ") or (im_crop_small_path[j,i] == ""): continue;
        else:
            for k in range(0,im_wavelet_paths.shape[2]):
                imageName = im_wavelet_paths[j,i,k]
                im = cv2.imread(imageName,0)
                mask = np.zeros((int(im.shape[0]),int(im.shape[1])))
                mask[1:-1,1:-1] = 1
                #plt.imshow(mask)
                mask_paths[j,i,k] = out_path + str(patients[i]) + "/wavelets/masks/L" + str(j+1) + "_mask_wavelet_" + str(k) + "_eq.png"
                maskName = mask_paths[j,i,k]
                cv2.imwrite(mask_paths[j,i,k],mask)

                if imageName is None or maskName is None:  # Something went wrong, in this case PyRadiomics will also log an error
                    print('Error getting testcase!')
                    exit()

                # Regulate verbosity with radiomics.verbosity (default verbosity level = WARNING)
                radiomics.setVerbosity(logging.INFO)

                # Get the PyRadiomics logger (default log-level = INFO)
                logger = radiomics.logger
                logger.setLevel(logging.INFO)
                logger.setLevel(logging.DEBUG)  # set level to DEBUG to include debug log messages in log file

                # Set up the handler to write out all log entries to a file
                handler = logging.FileHandler(filename='testLog.txt', mode='w')
                formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")
                handler.setFormatter(formatter)
                logger.addHandler(handler)

                # Define settings for signature calculation
                # These are currently set equal to the respective default values
                settings = {}
                settings['binWidth'] = 25
                settings['resampledPixelSpacing'] = None  # [3,3,3] is an example for defining resampling (voxels with size 3x3x3mm)
                settings['interpolator'] = sitk.sitkBSpline

                # Initialize feature extractor
                extractor = featureextractor.RadiomicsFeatureExtractor(**settings)

                # By default, only original is enabled. Optionally enable some image types:
                extractor.enableImageTypes(Original={}, LoG={}, Wavelet={})

                # Disable all classes except firstorder
                #extractor.disableAllFeatures()

                # Enable all features in firstorder
                # extractor.enableFeatureClassByName('firstorder')

                # Only enable mean and skewness in firstorder
                #extractor.enableFeaturesByName(firstorder=['Mean', 'Skewness'])
                #print("Calculating features", l)
                result = extractor.execute(str(imageName),str(maskName))
                result.pop("diagnostics_Versions_PyRadiomics")
                result.pop("diagnostics_Versions_Numpy")
                result.pop("diagnostics_Versions_SimpleITK")
                result.pop("diagnostics_Versions_PyWavelet")
                result.pop("diagnostics_Versions_Python")
                result.pop("diagnostics_Configuration_Settings")
                result.pop("diagnostics_Configuration_EnabledImageTypes")
                result.pop("diagnostics_Image-original_Hash")
                result.pop("diagnostics_Image-original_Dimensionality")
                result.pop("diagnostics_Image-original_Spacing")
                result.pop("diagnostics_Image-original_Size")
                result.pop("diagnostics_Mask-original_Hash")
                result.pop("diagnostics_Mask-original_Size")
                result.pop("diagnostics_Mask-original_Spacing")
                result.pop("diagnostics_Mask-original_BoundingBox")
                result.pop("diagnostics_Mask-original_VoxelNum")
                result.pop("diagnostics_Mask-original_VolumeNum")
                result.pop("diagnostics_Mask-original_CenterOfMassIndex")
                result.pop("diagnostics_Mask-original_CenterOfMass")
                #for featureName in result.keys():
                #    print("Computed %s: %s" % (featureName, result[featureName]))
                df = pd.DataFrame.from_dict(data=result, orient='index')
                df = df.T
                df["index"] = l
                df.set_index("index")
                df["Patient"] = str(patients[i])
                df["Vertebrae"] = "L" + str(j)
                df["Wavelet"] = str(k)
                df_test = pd.concat([df_test,df])
excel_path = out_path + "profile_wavelet_features.xlsx"
df_test.to_excel(excel_path,columns=cols)

In [ ]:
# Compute vertebral heights, wall lengths, plate angles for a vertain vertebral type 
# this version of the code only works with the raw data since it requires the non-cropped DRR images
# This cell needs to be rerun for each type of vertebra

"""
Task: Detect corners and fix perspective
Source: https://stackoverflow.com/questions/64860785/opencv-using-canny-and-shi-tomasi-to-detect-round-corners-of-a-playing-card
"""

def dist(p, q):
    #Return the Euclidean distance between points p and q.
    return math.hypot(p[0] - q[0], p[1] - q[1])
        
def horizontal_angle(p1,p2):
    xDiff = p2[0] - p1[0]
    yDiff = p2[1] - p1[1]
    return math.degrees(math.atan2(yDiff, xDiff))

anterior_heights = np.zeros(len(profile_L1_paths))
posterior_heights = np.zeros(len(profile_L1_paths))
length_top = np.zeros(len(profile_L1_paths))
length_left = np.zeros(len(profile_L1_paths))
length_bottom = np.zeros(len(profile_L1_paths))
length_right = np.zeros(len(profile_L1_paths))
top_plate_angles = np.zeros(len(profile_L1_paths))
bottom_plate_angles = np.zeros(len(profile_L1_paths))

for k in range(0,len(profile_L1_paths)):
    path = profile_L1_paths[k] #modify L1 to desired vertebra
    #print(path,flush="True")
    if path != ' ':
        mask = cv2.imread(path,0)
        gray = np.float32(mask)
        img = mask.astype(np.uint8)
        img = scipy.ndimage.rotate(img, -35, cval=0)
        gray = gray.astype(np.uint8)
        gray = scipy.ndimage.rotate(gray, -35, cval=0)
        # cut rectangle that fits mask
        x,y,w,h = cv2.boundingRect(img)
        img = img[y:y+h, x:x+w]
        plt.imshow(img)

        M = img.shape[0]#//2
        N = img.shape[1]#//2
        tiles = [img[x:x+M,y:y+N] for x in range(0,img.shape[0],M) for y in range(0,img.shape[1],N)]
        corners = np.zeros((4,2))

        for i in range(0,len(tiles)):
            im = tiles[i]
            gray = tiles[i]
            ret,thresh = cv2.threshold(gray,127,255,0)

            #cv2.imshow('Thresholded original',thresh)
            #cv2.waitKey(0)

            ## Get contours
            contours,h = cv2.findContours(thresh,cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

            ## only draw contour that have big areas
            imx = im.shape[0]
            imy = im.shape[1]
            lp_area = (imx * imy) / 10

            ## Get only rectangles given exceeding area
            for cnt in contours:
                approx = []
                constant = 0.01
                while len(approx) != 4 and constant <= 1:
                    approx = cv2.approxPolyDP(cnt,constant * cv2.arcLength(cnt, True), True)
                    ## calculate number of vertices
                    #print(len(approx))
                    constant = constant + 0.01

                if constant == 1: print("Could not approximate polygon.")
                else:
                    #print("rectangle")

                    tmp_img = im.copy()
                    cv2.drawContours(tmp_img, [cnt], 0, (0, 255, 255), 6)
                    #cv2.imshow('Contour Borders', tmp_img)
                    #cv2.waitKey(0)


                    tmp_img = im.copy()
                    cv2.drawContours(tmp_img, [cnt], 0, (255, 0, 255), -1)
                    #cv2.imshow('Contour Filled', tmp_img)
                    #cv2.waitKey(0)


                    # Make a hull arround the contour and draw it on the original image
                    tmp_img = im.copy()
                    mask = np.zeros((im.shape[:2]), np.uint8)
                    hull = cv2.convexHull(cnt)
                    cv2.drawContours(mask, [hull], 0, (255, 255, 255), -1)
                    #cv2.imshow('Convex Hull Mask', mask)
                    #cv2.waitKey(0)


                    # Draw minimum area rectangle
                    tmp_img = im.copy()
                    rect = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    cv2.drawContours(tmp_img, [box], 0, (0, 0, 255), 2)
                    #cv2.imshow('Minimum Area Rectangle', tmp_img)
                    #cv2.waitKey(0)


                    # Draw bounding rectangle
                    tmp_img = im.copy()
                    x, y, w, h = cv2.boundingRect(cnt)
                    cv2.rectangle(tmp_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    #cv2.imshow('Bounding Rectangle', tmp_img)
                    #cv2.waitKey(0)


                    # Bounding Rectangle and Minimum Area Rectangle
                    tmp_img = im.copy()
                    rect = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    cv2.drawContours(tmp_img, [box], 0, (0, 0, 255), 2)
                    x, y, w, h = cv2.boundingRect(cnt)
                    cv2.rectangle(tmp_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    #cv2.imshow('Bounding Rectangle', tmp_img)
                    #cv2.waitKey(0)


                    # determine the most extreme points along the contour
                    # https://www.pyimagesearch.com/2016/04/11/finding-extreme-points-in-contours-with-opencv/
                    tmp_img = im.copy()
                    extLeft_tmp = tuple(cnt[cnt[:, :, 0].argmin()][0])
                    extRight_tmp = tuple(cnt[cnt[:, :, 0].argmax()][0])
                    extTop_tmp = tuple(cnt[cnt[:, :, 1].argmin()][0])
                    extBot_tmp = tuple(cnt[cnt[:, :, 1].argmax()][0])
                    #print("Corner Points: ", extLeft_tmp, extRight_tmp, extTop_tmp, extBot_tmp)

                    corners[0][0] = extLeft_tmp[0]
                    corners[0][1] = extLeft_tmp[1]
                    corners[1][0] = extTop_tmp[0]
                    corners[1][1] = extTop_tmp[1]
                    corners[2][0] = extRight_tmp[0]
                    corners[2][1] = extRight_tmp[1]
                    corners[3][0] = extBot_tmp[0]
                    corners[3][1] = extBot_tmp[1]
                    cv2.circle(tmp_img, extLeft_tmp, 8, (128, 128, 128), -1)
                    cv2.circle(tmp_img, extRight_tmp, 8, (128, 128, 0), -1)
                    cv2.circle(tmp_img, extTop_tmp, 8, (128, 128, 128), -1)
                    cv2.circle(tmp_img, extBot_tmp, 8, (128, 128, 128), -1)


                    #cv2.imshow('img contour drawn', tmp_img)
                    #cv2.waitKey(0)
                    #cv2.destroyAllWindows()
        tmp_img = im.copy()
        for x,y in corners:
            cv2.circle(tmp_img, (int(x),int(y)), 8, (128, 128, 128), -1)

        plt.figure(i+1)
        plt.imshow(tmp_img,cmap="gray")
        plt.show()
        #axs[i].imshow(tmp_img,cmap="gray")
        #axs[i].set_axis_off()
        #axs[i].set_title(i)
        anterior = np.asarray([corners[0,:],corners[1,:]])
        print(anterior)
        posterior = np.asarray([corners[2,:],corners[3,:]])
        print(posterior)
        # Compute anterior and posterior distances
        # According to EOS imaging, pixel size = 254 μm --> 0.18 mm
        
        anterior_heights[k] = dist(anterior[0,:], anterior[1,:])*0.18
        print("Anterior height (mm):", anterior_heights[i])
        posterior_heights[k] = dist(posterior[0,:], posterior[1,:])*0.18
        print("Posterior height (mm):", posterior_heights[i])
        length_top[k] = dist(corners[1,:], corners[2,:])*0.18
        length_left[k] = dist(corners[0,:], corners[1,:])*0.18
        length_bottom[k] = dist(corners[3,:], corners[0,:])*0.18
        length_right[k] = dist(corners[2,:], corners[3,:])*0.18
        #cv2.destroyAllWindows()
        top_plate_angles[k] = horizontal_angle(anterior[1,:],posterior[0,:])-35
        bottom_plate_angles[k] = horizontal_angle(anterior[0,:],posterior[1,:])-35
    else: continue;
        
df = pd.DataFrame({'Patient': patients,'Anterior Height': anterior_heights, 'Posterior Height': posterior_heights, 'Top Plate Angle': top_plate_angles, 'Bottom Plate Angle': bottom_plate_angles, 'Length Top': length_top, 'Length Right': length_right, 'Length Bottom': length_bottom, 'Length Left': length_left})
df.to_excel(out_path + "profile_heights_angles_lengths_L1.xlsx") # modify name to match vertebra analyzed

In [ ]:
# ISV calculation
# Modified code from:
# Maquer, Ghislain & Lu, Yongtao & Dall'Ara, Enrico & Chevalier, Yan & Krause, Matthias & Yang, Lang & Eastell, Richard & Lippuner, Kurt & Zysset, Philippe. (2017). ISV: python implementation and example. 

#Functions
# ... bilinear interpolation
def interpolation(im, x, y):
    x   = np.asarray(x); y = np.asarray(y)
    x1  = int(np.floor(x)); y1 = int(np.floor(y)); x2 = x1 + 1; y2 = y1 + 1 # positions around (x,y)
    x1  = np.clip(x1, 0, im.shape[0]-1); y1 = np.clip(y1, 0, im.shape[1]-1); x2 = np.clip(x2, 0, im.shape[0]-1); y2 = np.clip(y2, 0, im.shape[1]-1); # limit interpolation to the image's pixels
    I11 = im[ x1, y1 ]; I12 = im[ x1, y2 ]; I21 = im[ x2, y1 ]; I22 = im[ x2, y2 ] # intensity of the pixels around
    w11 = (x2-x)*(y2-y) ; w12 = (x2-x)*(y-y1) ; w21 = (x-x1)*(y2-y) ; w22 = (x-x1)*(y-y1) # weights according to the distance of (x,y) to the pixels around
    points = (w11*I11 + w12*I12 + w21*I21 + w22*I22) / ((x2-x1)*(y2-y1))
    if np.isnan(points): points = 0
    if math.isinf(points): points = 0
    return points

# ... rotate a point around another
def rotatePoint(rotationcenter,originalpoint,angle):
    angle         = math.radians(angle)
    rotatingpoint = originalpoint[0]-rotationcenter[0] , originalpoint[1]-rotationcenter[1]
    rotatingpoint = ( rotatingpoint[0]*math.cos(angle)-rotatingpoint[1]*math.sin(angle) , rotatingpoint[0]*math.sin(angle)+rotatingpoint[1]*math.cos(angle)) # rotation counter-clockwise
    rotatingpoint = rotatingpoint[0]+rotationcenter[0] , rotatingpoint[1]+rotationcenter[1]
    return rotatingpoint

# ... isotropic semi-variogram optimized function
def SV(P,h,neig,sampleROI):
    stdout.write( "     Lag %i... \n" %(h) ); stdout.flush()
    u0 = np.asarray([1, 0]) # the unit vector in the direction 0 deg (horizontal)
    a = 1
    b = 1
    SVar = np.zeros((neig,len(sampleROI)))
    for theta in np.random.uniform(0.0, 360.0, size=neig):
        # compute the semivariance of each P(i,j) and its random neighbours at distance h if they are in the ROI
        for b in range(0,len(sampleROI)):
            i = sampleROI[b][0]
            j = sampleROI[b][1]
            interp = interpolation( P, rotatePoint((i,j),(i+h*u0[0],j+h*u0[1]),theta)[0], rotatePoint((i,j),(i+h*u0[0],j+h*u0[1]),theta)[1] )
            if interp !=0:
                SVar[a-1][b-1] = np.mean(P[i,j]- interp)**2
            b=b+1
        a=a+1
    return np.mean(SVar) # average semivariance over all possible pairs

# ISV computation
### ... parameter entry
#Type             argId    Description                               Default           Opt.  Info 
guiInit = "*modulName     'Initial slope of the variogram'                                                     \n"\
+"*subWindowStart          'Initial slope of the variogram'                                                     \n"\
+"*fileEntryIn    -in      'Input file'                               filename.mhd      no    mhd               \n"\
+"*entry          -pix     '(%) Fraction of the region of interest'   100               yes   1                 \n"\
+"*entry          -hmax    'Number of lags'                           6                 yes   1                 \n"\
+"*entry          -neig    'Number of neighbours'                     36                yes   1                 \n"\
+"*entry          -pts     'Number of points for slope computation'   5                 yes   1                 \n"\
+"*subWindowEnd            'Initial slope of the variogram computed'                                                     \n"

ISVs = np.zeros((4,len(profile_L1_paths)))
rs = np.zeros((4,len(profile_L1_paths)))
p = np.zeros(len(profile_L1_paths))
for k in range(40,len(profile_L1_paths)):
    p = patients[k]
    for j in range(0,4):
        path = im_profile_crop_small_path[j,k]
        #print(path,flush="True")
        if path != '':
            infile = '%s' % path #im_crop_small_path # in
            #outfile = "filename.pdf" # out
            pix = 25 # pix
            hmax = 6 # hmax
            #neig = 36 # neig
            #pts = 5 # pts
            #txt = "filename.txt" # txt
            #mode = "w" # mode

            argList = argv
            argc = len(argList)

            # ... Default values for the optional arguments
            stdout.write( "\n ... any default parameters? \n" ); stdout.flush()
            try: print(pix)
            except NameError: print("     pix: default value 100"); pix  = 100

            try: print(hmax)
            except NameError: print("     hmax: default value 6"); hmax  = 6

            try: print(neig)
            except NameError: print("     neig: default value 36"); neig = 36

            try: print(pts)
            except NameError: print("     pts: default value 6"); pts  = 5

            # start the code
            # ... get name
            path, name = os.path.split(infile)

            # ... read image
            stdout.write( "\n ... reading image \n" ); stdout.flush()
            im      = sitk.ReadImage(infile)
            #im = Image.fromarray(im)
            ndim = im.GetSize()
            print(ndim)
            array   = sitk.GetArrayFromImage(im)

            # ... compute semi-variogram
            stdout.write( "\n ... building the variogram \n" ); stdout.flush()
            if hmax < 6:  hmax = 6; stdout.write( "  ! number of lags too low was set to 6 ! \n" ); stdout.flush()
            if   ndim[0] == 1: P  = array[:,:,0]; # data in plane (Y, Z)
            elif ndim[1] == 1: P  = array[:,0,:]; # data in plane (X, Z)
            #elif ndim[2] == 1: P  = array[0,:,:]; # data in plane (X, Y)
            else: P = array;
            dimx      = P.shape[0]; dimy = P.shape[1]; 
            pixROI    = [(i,j) for i in range(dimx) for j in range(dimy) if P[i,j]!=0]
            sampleROI = random.sample(pixROI, int(len(pixROI)*pix/100.0)) 
            hs = np.asarray( np.arange(1,hmax+1) ) # lags
            stdout.write( "     %i pixels in the image \n" %(ndim[0]*ndim[1]) ); stdout.flush()
            stdout.write( "     %i pixels in the region of interest \n" %(len(pixROI)) ); stdout.flush()
            stdout.write( "     %i pixels randomly selected from the region of interest \n" %(len(sampleROI)) ); stdout.flush()
            stdout.write( "     %i neighbours randomly selected for each pixel \n" %(neig) ); stdout.flush()
            stdout.write( "     %i lags for the computation \n" %(hmax) ); stdout.flush()
            sv = np.asarray( [SV(P,h,neig,sampleROI) for h in hs] )

            # ... compute ISV
            stdout.write( "\n ... computation of the ISV \n" ); stdout.flush()
            if pts >= hmax: pts = hmax-1; stdout.write( " ! number of points asked for the slope calculation exceeds the number of lags ! \n" ); stdout.flush()
            if pts < 2: pts = 2; stdout.write( " ! a slope cannot be calculated with a single point ! \n" ); stdout.flush()
            stdout.write( " %i points for the slope calculation \n" %(pts) ); stdout.flush()
            ISV, Nugget, r, p, std_err = scipy.stats.linregress( np.log10( hs[:pts] ), np.log10 (sv[:pts] ) )
            ISV = round(ISV,3)
            ISVs[j,k] = ISV
            rs[j,k] = r
            stdout.write( " -> Initial slope of the variogram = %0.3f\n" %(ISV) ); stdout.flush()
            stdout.write( " -> Goodness of fit of the slope = %0.3f\n" %(r**2) ); stdout.flush()

        else: continue;
        
df = pd.DataFrame({'Patients': p,'ISV L1': ISVs[0,:], 'ISV L2': ISVs[1,:], 'ISV L3': ISVs[2,:], 'ISV L4': ISVs[3,:], 'Goodness of fit (R) L1': rs[0,:], 'Goodness of fit (R) L2': rs[1,:], 'Goodness of fit (R) L3': rs[2,:], 'Goodness of fit (R) L4': rs[3,:]})
df.to_excel(out_path + "profile_ISV_features.xlsx")